In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import pandas as pd 
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt 
from sklearn.ensemble import GradientBoostingRegressor
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
%matplotlib inline 
import numpy as np
plt.rcParams['figure.figsize'] = (16, 4)

import warnings
warnings.filterwarnings(action="ignore")
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

from utils import * # We've added all the functions from the last BLU to the utils.py 

# DATA

In [2]:
df = pd.read_csv("../data/data_v2.csv", index_col="time", parse_dates=True)

In [3]:
df = build_exog_features(df, 24*7)

In [4]:
df["baseline"] = df.price.shift(24*7)

In [5]:
train = df.iloc[:-24*7*2]
val = df.iloc[-24*7*2:-24*7]
train_val = df.iloc[:-24*7]
test = df.iloc[-24*7:]

# BASELINE

In [6]:
val_mae = mean_absolute_error(val["price"],val["baseline"])
print(val_mae)

5.729642857142856


# VALIDATION

In [7]:
predictions = predict_n_periods(series_=train, 
                  n_periods=24*7, 
                  model=LinearRegression(), 
                  num_periods_lagged=24*7,
                  num_periods_diffed=0,
                   weekday = True,
                  month = True,
                  holidays = False,
                  target = "price"
                  )

  0%|          | 0/168 [00:00<?, ?it/s]

In [8]:
val_mae = mean_absolute_error(val["price"],predictions)
print(val_mae)

4.107383709227124


In [9]:
val["predictions"] = predictions

In [10]:
val[["predictions","price"]].plot()

# TEST

In [12]:
labels = pd.read_csv("test_labels.csv")
labels.index = test.index

In [13]:
labels["baseline"] = df.baseline
test_mae = mean_absolute_error(labels["price"],labels["baseline"])
print(test_mae)

19.07803571428571


In [14]:
predictions = predict_n_periods(series_=train_val, 
                  n_periods=24*7, 
                  model=LinearRegression(), 
                  num_periods_lagged=24*7,
                  num_periods_diffed=0,
                   weekday = True,
                  month = True,
                  holidays = False,
                  target = "price"
                  )

  0%|          | 0/168 [00:00<?, ?it/s]

In [15]:
labels["predictions"] = predictions

In [16]:
labels[["predictions","price","baseline"]].plot()

In [17]:
test_mae = mean_absolute_error(labels["price"],predictions)
print(test_mae)

12.71004357575448
